# Curs 6: Optimizarea modelelor, preprocesare, pipelines

## Optimizarea modelelor

In cursul anterior s-a aratat cum se poate folosi k-fold cross validation pentru estimarea performantei unui model. Totodata, s-a aratat o maniera simpla de cautare a valorilor celor mai potrivite pentru hiperparametri - in cazul respectiv. valoarea adecvata a numarului de vecini. 

Vom continua aceasta idee pentru mai multi hiperparametri, apoi folosim facilitatile bibliotecii `sklearn` pentru automatizarea procesului.

K-fold cross validation ( asigura ca fiecare din cele k partitii ale setului de date initial este pe rand folosit ca subset de testare:

In [ ]:
from sklearn.model_selection import KFold
!pip install prettytable --upgrade
from prettytable import PrettyTable

In [ ]:
kf = ...
splits = ...
t = PrettyTable(['Iter', 'Train', 'Test'])
t.align = 'l'
for i, data in enumerate(splits):
    t.add_row([i+1, data[0], data[1]])
print(t)

Pentru calculul performantei prin k-fold CV se asigura ca esantionarea se face in mod stratificat: fiecare fold are aceeasi proportie a claselor ca si in setul originar.

Folosim k-fold cross validation pentru a face evaluarea de modele pentru diferite valori ale hiperparametrilor. 

In [ ]:
import numpy as np
import pandas as pd
print ('numpy: ', np.__version__)
print ('pandas: ', pd.__version__)

In [ ]:
from sklearn.model_selection import ...
from sklearn.neighbors import ...
from sklearn.metrics import ...

from sklearn.datasets import load_iris

iris = ...
X = ...
y = ...

Pentru k-nearest neighbors vom cauta valorile optime pentru:
* numarul de vecini, $k \in \{1, \dots, 31\}$
* putere corespunzatoare metricii Minkowski:
$$
d(\mathbf{x}, \mathbf{y}) = \left( \sum\limits_{i=1}^n \left|x_i-y_i\right|^p \right)^{1/p}
$$

In [ ]:
best_score = 0
for k in range(1, 31):
    for p in [1, 2, 3, 4.7]:
        ...
print('Best score:', best_score)
print('Best params:', best_params)   
model = ...
model.fit(X, y)
y_predicted = model.predict(X)
print('Accuracy on whole set:', accuracy_score(y, y_predicted))

Pentru procesul de mai sus urmatoarele comentarii sunt necesare:
1. strategia implementata se numeste grid search: se cauta peste toate combinatiile de 30\*4 variante si sa retine cea mai buna; este consumatoare de resurse, dar o prima varianta de lucru acceptabila
1. am dori sa avem o modalitate automatizata de considerare a tuturor combinatiilor de parametri din multimea de valori candidat. Codul devine greu de scris cand sunt multi hiperparametri, fiecare cu multimea proprie de valori candidat
1. estimarea efectuata in final este de cele mai multe ori optimista: optimizarea parametrilor s-a facut peste niste date, care date in final sunt cele folosite pentru evaluarea finala; am ajuns practic sa facem evaluare pe setul de antrenare, ceea ce e o idee proasta. Estimarea finala a performantelor modelului trebuie facuta peste un set de date aparte, care nu a fost folosit nici pentru antrenare, nici pentru validarea modelelor candidat. 


Pentru ultimul punct se recomanda ca setul sa fie impartit ca mai jos:
![train_validation_test](./images/train_validation_test.png)

Ca atare, va trebui sa rescriem codul astfel:

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=1/5)
...
print('Best score:', best_score)
print('Best params:', best_params)   

model = ...
model.fit...
y_predicted = model.predict(X_test)
print(accuracy_score(y_test, y_predicted))

Desigur, si implementarea de mai sus e criticabila: s-a facut evaluare pe un singur set de testare, anume cel rezultat dupa impartirea initiala in partitiile \*\_trainvalid si \*\_test. Este totusi o estimare mai corect facuta decat cea precedenta.  In realitate, acest stil de lucru este frecvent intalnit: exista un set de testare unic, dar necunoscut la inceput. Singurele date disponibile sunt impartite in *training set* si *validation set* (eventual mai multe) pentru a obtine un model care se spera ca generalizeaza bine = se comporta bine pe setul de testare. 

Varianta anterioara se numeste **grid search with cross validation**. Exista clasa `sklearn.model_selection.GridSearchCV` care automatizeaza procesul:


In [ ]:
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/5)
...

In [ ]:
y_estimated = grid_search.predict(X_test)
print(accuracy_score(y_test, y_estimated))

Valorile optimale ale hiperparametrilor sunt retinute in atributul `best_params_`:

In [ ]:
print(grid_search.best_params_)

In codul anterior denumirile cheilor din dictionarul `parameter_grid` nu sunt intamplatoare: ele coincid cu numele parametrilor modelului vizat. Instantierea `estimator = KNeighborsClassifier()` se face cu valorile implicite ale parametrilor, apoi insa se ruleaza metode de tip `set_` care seteaza parametrii dati in dictionarul `parameter_grid`.

Pentru cei interesati, valorile de performanta pentru fiecare fold se pot inspecta in campul `cv_results_`. Pentru ca acestea sa fie disponibile, este obligatorie setarea parametrului `return_train_score=True` din clasa `GridSearchCV`.

In [ ]:
...

In [ ]:
df_grid_search.head()

Pentru situatia in care se doreste evaluarea nu doar pe un singur set de testare, ci in stil cross-validation, se poate face un *nested cross-validation*:

In [ ]:
scores = ...

In [ ]:
print(scores.mean())

## Metode de preprocesare

Uneori, inainte de aplicarea vreunui model, este nevoie ca datele de intrare sa fie supuse unor transformari. De exemplu, daca pentru algoritmul k-NN vreuna din trasaturi (fie ea *F*) are valori de ordinul sutelor si celelalte de ordinul unitatilor, atunci distanta dintre doi vectori ar fi dominata de diferenta pe dimensiunea *F*; celelalte dimensiuni nu ar conta prea mult.

Intr-o astfel de situatie se recomanda sa se faca o scalare in prealabil a datelor la intervale comparbile, de ex [0, 1]. 

In modulul `sklearn.preprocessing` se afla clasa `MinMaxScaler` care permite scalarea independenta a trasaturilor. Il vom demonstra pe un set de date care are trasaturi cu marimi disproportionate.

In [ ]:
from sklearn.datasets import load_breast_cancer
...

In [ ]:
def print_ranges(X):
    ...
        
print_ranges(X)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
...
print_ranges(X)

De mentionat ca secventa `fit` si `transform` se poate apela intr-un singur pas:

In [ ]:
X, y = medical.data, medical.target
...

De regula, setul de date se imparte in doua (in modul naiv): set de antrenare si set de testare. Se presupune ca setul de testare este cunoscut mult mai tarziu decat cel de antrenare. Ca atare, doar cel de antrenare se trece prin preprocesor, iar  valorile 'invatate' via `fit ` se pastreaza (obiectul de tip `MinMaxScaler` are stare). Ele vor fi folosite pentru scalarea setului de test:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)
scaler = MinMaxScaler()
...

Se remarca faptul ca, folosindu-se parametrii de scalare din setul de antrenare, nu se poate garanta ca setul de testare este cuprins de asemenea in hipercubul unitate $[0, 1]^{X.shape[1]}$

Exista si alte metode de preprocesare in modulul [`sklearn.preprocessing`](http://scikit-learn.org/stable/modules/preprocessing.html). 


## Pipelines

Se prefera inlantuirea intr-un proces a pasilor: preprocesare si aplicare de model. Exemplificam pentru cazul simplu in care exista un set de antrenare si unul de testare:

In [ ]:
X, y = medical.data, medical.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [ ]:
from sklearn.pipeline import Pipeline
...

Pentru cazul in care se vrea k-fold cross validation pentru determinarea valorilor optime pentru hiperparametri, urmata de testare pe un set de testare:

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=1/3)
parameter_grid = {'knn__n_neighbors': list(range(1, 10)), 'knn__p': [1, 2, 3, 4.7]}
grid = GridSearchCV(pipe, param_grid = parameter_grid, scoring = 'accuracy', cv=5)
grid.fit(X_trainval, y_trainval)

In [ ]:
y_predicted = ...